# LLM Stack Hackathon (June 3, 2023) Starter Kit

## Setup

In [1]:
import os
import json

import openai

from dotenv import load_dotenv
from IPython.display import HTML, display
import pandas as pd
from IPython.display import HTML, display
from tqdm import tqdm
import numpy as np

import redis
from redis.commands.search.field import TagField, VectorField, TextField
from redis.commands.search.indexDefinition import IndexDefinition, IndexType
from redis.commands.search.query import Query

load_dotenv()
openai.organization = os.getenv("OPENAI_ORG")
openai.api_key = os.getenv("OPENAI_API_KEY")
# Point to your self-hosted redis server OR try a free redis cloud server from https://redis.com/try-free/
REDIS_URL = os.getenv("REDIS_URL")

try:
    openai.Model.list()
except:
    print("OpenAI authentication error")


In [2]:
# Since we have some long lines, let's make sure output is line wrapped
def set_css():
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))
get_ipython().events.register('pre_run_cell', set_css)

## Download the data

Data already download in ~/data folder.

In [3]:
# import gdown
# gdown.download_folder("https://drive.google.com/drive/folders/1FCuU2j8yI7hXsZL8Ls_fgJwUn7-Dx_VV?usp=sharing", quiet=False)

### Raw Messages & Embeddings

In [4]:
# Raw messages
messages_df = pd.read_csv("../data/messages.csv")
messages_df.head()

,__Source,User_ID,Channel_Name,Message_Timestamp,Thread_Timstamp,Channel_ID,__Text
0,threads,U01T78HPG3H,computer-vision,2023-05-18 11:36:44.001949 UTC,2023-05-18 11:30:13.159979 UTC,C026ED0PZEZ,Both <https://roboflow.github.io/supervision/q...
1,threads,U01T78HPG3H,computer-vision,2023-05-18 11:30:13.159979 UTC,2023-05-18 11:30:13.159979 UTC,C026ED0PZEZ,Would love a package for suggesting and then i...
2,threads,U04QRD69H8Q,computer-vision,2023-05-18 10:56:33.313369 UTC,2023-05-17 12:35:14.522419 UTC,C026ED0PZEZ,<@U056Q4V4FFC> how about this dataset (for the...
3,threads,U01J0NVNE1G,mlops-questions-answered,2023-05-18 05:21:04.346819 UTC,2023-05-18 01:08:57.948139 UTC,C015J2Y9RLM,"I always oppose the counterargument, why do yo..."
4,threads,U01J0NVNE1G,mlops-questions-answered,2023-05-18 05:19:51.718379 UTC,2023-05-18 01:08:57.948139 UTC,C015J2Y9RLM,I find MLflow more convenient to use. Here are...


In [5]:
# Get all data for specific channel
df_mlops_questions_answered = messages_df[messages_df["Channel_Name"]=="mlops-questions-answered"]
df_mlops_questions_answered.head()

,__Source,User_ID,Channel_Name,Message_Timestamp,Thread_Timstamp,Channel_ID,__Text
3,threads,U01J0NVNE1G,mlops-questions-answered,2023-05-18 05:21:04.346819 UTC,2023-05-18 01:08:57.948139 UTC,C015J2Y9RLM,"I always oppose the counterargument, why do yo..."
4,threads,U01J0NVNE1G,mlops-questions-answered,2023-05-18 05:19:51.718379 UTC,2023-05-18 01:08:57.948139 UTC,C015J2Y9RLM,I find MLflow more convenient to use. Here are...
5,threads,U01CRVDS4NA,mlops-questions-answered,2023-05-18 05:14:52.514189 UTC,2023-05-16 23:22:04.332479 UTC,C015J2Y9RLM,I just built some demos with it. The developer...
7,messages,U01VCA57PD0,mlops-questions-answered,2023-05-18 01:08:57.948139 UTC,2023-05-18 01:08:57.948139 UTC,C015J2Y9RLM,These days I'm feeling very tempted to roll my...
21,messages,U015BH45ZK6,mlops-questions-answered,2023-05-17 14:56:59.775629 UTC,2023-05-17 14:56:59.775629 UTC,C015J2Y9RLM,I ran into a problem downloading files from Az...


In [6]:
# Get the count of conversations
print(f"Number of conversations: {len(df_mlops_questions_answered.groupby(['Thread_Timstamp']))}")

Number of conversations: 2086


In [7]:
# Iterate through conversations
NUM_CONVERSATIONS_TO_PRINT=5
current_conversation=0
for (channel_name, thread_id), conv in df_mlops_questions_answered.groupby(['Channel_Name', 'Thread_Timstamp']):
  html = f"<h2>Channel: {channel_name} / conversation {thread_id} </h2><hr/>\n<table>\n"
  for index, row in conv.iterrows():
    html+= f'<tr><td>{row["User_ID"]}</td><td>{row["__Text"]}</td></tr>'
  html += f"</table>\n<hr/><br/>"
  display(HTML(html))
  current_conversation+=1
  if current_conversation>=NUM_CONVERSATIONS_TO_PRINT:
    break

U011NTHUKEF,"We use a lot of TPOT library. The main advantage is that it does hyperparameter tunning and also deals with preprocessing steps as well... downside is that it works mostly with scikit-learn algorithms. But there is a small hack you can do to make it work with many algorithms... make a new algo class that inherits both from sklearn BaseEstimators and your algo (Catboost, for example, or Pygams)"
U015CHWG25B,"Aha, I’ve heard that it’s really good!"
U0150LZ578X,"For anyone already using other Kubeflow components, Katib is relatively easy to work with It parallelizes trials across k8s pods and provides a web UI to visualize the hyperparameter space for each training history!"
U015CHWG25B,<@U013CL3GTB3> Thank you! Looks like this is an entire ML platform rather than just a hyperparameter tuning tool :thinking_face:
U013CL3GTB3,Polyaxon is another good one!
U015CHWG25B,"At some point we reviewed about 15 hyperparameter tuning tools in order to choose one that answers our needs. We stopped at NNI from Microsoft (). This tool is designed to run hyperparameter tuning in several parallel jobs. Unlike many other tools, it supports a lot of different algorithms of hyperparameter tuning. It has a decent UI. Plus, it's OSS from Microsoft :) Do you know other tools which answer our criteria?"
U015CHWG25B,I want to run hyperparameter tuning on several machines in parallel and track the process in Web UI. What is the best tool for that?


U011NTHUKEF,"AirFlow is a great tool. We use it a lot. When we work on a AWS environment, we use AWS stepfunctions which has its own Data Science SDK. Works like a charm!"
U015CHWG25B,"This is a clear usecase for pipeline tools. There are plenty of them out there, providing various features and UI capabilities. Currently, for our projects we use AirFlow. Which pipeline tools do you prefer and why?"
U015CHWG25B,"OK, I have a model of decent quality. Now I want to automate daily collecting data, retraining the model, and redeployment. How do I do that?"


U015CHWG25B,"Sure, thank you! :pray:"
U016A3RAL5N,<@U015CHWG25B> thanks for including us in your list. If you need any support lest us know


UV92GMLF4,"<@U014Z58NT25> Sorry for the late reply: 1. It's determined by marketing. The thing that we do it's just to highlight who is in the border. 2. We used only numericals in that time, but OHE work fine also. 3. At the that time, we didn't used any specific tool. I think today it's doable to do in matplotlib if you transform in 2D array those records from the cluster."
U012YQULW4X,"you seem to treat it as a technical problem (automate the encoding). This issue points you to a potentially significant change in the input data so you want to notice and manually investigate. For detection, you can check tools for data monitoring like or tensorflow data monitoring. They should be able to check feature cardinality (number of segments). Segments: Do I understand you correctly that the marketing team changed the logic for the segment variable e.g. an prior ""A"" customer might now be a ""B"" customer? Case A: Feature is not important. Fix encoding bug and move on. Case B: The feature is important but not a lot of customers migrated their segment. Fix encoding bug and move on. Case C: Feature is important and the logic is very different or many people migrated. This is possibly a breaking change in the data and you need a migration plan (discard training data with old logic, not use the feature for the migration duration etc): you mention: ""everytime this happens"". That should not happen regularly. If it does, you have to stop using the feature/understand better what they do and what it means. Constantly changing data definitions make the feature dangerous to use."
UP3T8K9M5,maybe <@U012YQULW4X> can also help as she has some experience with monitoring and will be talking to us about it during this week’s meetup
U014Z58NT25,And thanks for the answers guys! It really helps even just to talk about the problem
U014Z58NT25,"<@UV92GMLF4> the clusters were determined by the marketing in the sense of ""good cluster/bad cluster"" or they even gave you a centroid for that? Could you use anything but numericals for KMeans? Essentially I know you can't, for you have to deal with distances. But someone has once told me you can sometimes work with binary/ordinal using KMeans as well. Also, did you have anything (besides your own code and CLI) for monitoring those jumpers? Any specific tools or dashboards"
U014Z58NT25,"<@U013CL3GTB3> What happens is that as of now I have, say, 5 customer segments. On the data that I extract tomorrow, I might have 6, cause this customer segmentation is something still being created by the business, and I won't know it in advance. I can extract some meaning from these classes and put them into an ascending order (of integers, for example). And yes, I wanna use them for my recsys at the end of the day. EDIT: typo"
UP3T8K9M5,<@U0156CADGJG> might have something to say about this too
UV92GMLF4,"Those guys I removed from the clusters and put them in another cluster (determined by marketing). Was totally primitive and I was using the KMeans, but the general idea it's that."
UV92GMLF4,"In pink I got the ""jumpers""."
UV92GMLF4,Those are the clusters (just for simplification)
UV92GMLF4,"Hey Murilo! What's up! I worked with that in a long time ago (2015) so maybe my answer it's outdated. In my case, I had fixed clusters (due to the fact those clusters where determined by Marketing); so the only thing that I monitor in this case was 1) the Centroid (to check if there's some changes in the centroid (drift) and 2) in the instances that I call ""jumpers"", i.e. records that shifted to some point of clusters boarders. Like this."


U016FCTSDGS,"Few answers from our experience: 1. We keep Airflow stack separate from DAG Projects. We also have a single repo per DAG project that gets auto-named and deployed in such a way that Airflow picks it up to sync in the DAG folder. We also have a DAG that runs frequently to scan and ingest the new DAG(s) from the DAG Projects. 2. If the question is long running tasks we spin up AWS Batch or AWS EMR depending on the need. If the question is how to interject DAG updates if a long running task is executing then no solution here other than on the next run the task(s)/DAG will get updated. 3. Did not know about the *`PythonVirtualenvOperator`* or would have used it. We just installed custom python packages to a temp directory and added to Python path, then deleted the folder in the last task for cleanup. 4. Haven't used Docker Operator. 5. Let me know if you find one!"
U015CHWG25B,"I can’t answer all your questions one by one, but I can share the basics of our approach. • Regarding deployment: We use Git to deploy DAGs. In our case, we have a dedicated repo for DAGs, and AirFlow installation polls this repo every minute and redeploys DAGs if necessary. Thus your DAGs are separated from your Airflow service. • Regarding tasks: We chose to make our own operator (as we have an ML platform with some specificity) based on DockerOperator. Containerisation of each step gives you way more freedom, especially in complex pipelines when you may need even different versions of CUDA for different steps."
U013K7876BF,"thanks for the responses <@U013CL3GTB3> and <@U0158N59C8H> Flux sounds great, but I’m going need to work with a push model (we have some constraints in our system - we can’t access the repo from the cloud) If i’m pushing the scheduler just to kick off a `KubernetesPodOperator`, Airflow feels very similar to Argo."
UP3T8K9M5,hmmm yeah i also would like to know this.
U0158N59C8H,Nope. I mean I use argo quite a bit via kubeflow pipelines. I used airflow too. Just wanting to get more opinions on the diff solutions. Esp Argo vs Tekton. As they are very closely related.
UP3T8K9M5,I also remember <@U0158N59C8H> was asking about Argo a while back. Were you able to get any info that could help us?
U013CL3GTB3,"1. I would launch the task in a pod using the kubernetesPodOperator. And for pushing your dags, use gitops! You can point your airflow deployment to a git branch. This requires flux and some other stuff to be in place. 2. Not sure what you mean by long running airflow service? Are you saying how to update a task independent of airflow itself like the web server or scheduler? - if you're launching your tasks in pods you leave the execution logic to the docker container running in the pod and the orchestration logic to airflow. I would also recommend using the KubernetesExecutor alongside the KubernetesPodOperator. 3. The kubernetesPodOperator also solves this problem as you have complete control over the runtime environment, resources, etc - the container has everything it needs to run the code 4. Haven’t used the docker operator but It sounds like it’s more general than just python code. And will allow you to containerize your code which deals with the dependency issue. 5. My company BenevolentAI Is about to publish an article about some lessons we learned using airflow so I’ll post it to the community when it’s out. Also me and <@UP3T8K9M5> are planning on doing a coffee session on pipelines - and airflow will be one of them. You mentioned Argo, that’s also another good option as it’s native to kubernetes which is nice and also has other benefits."
U013K7876BF,"Hey everyone We’re looking for productionize some data engineering pipelines but we really want this run on Airflow and have a proper CI/CD pipeline. I have a couple of questions, and I’m hoping some of you may have experience and knowledge about these topics. 1. Would you deploy the Airflow task along with the Airflow service itself? How else would you push your DAG

In [8]:
# Embeddings data for each message (rahul is generating this rn.) 
message_embeddings_df = pd.read_csv("../data/messages-embeddings-ada-002.csv")
message_embeddings_df.head()

,Unnamed: 0,message_id,embedding
0,0,2020-03-17 22:52:45.0174 UTC,"[-0.043248970061540604, -0.007734753657132387,..."
1,1,2020-03-17 22:58:47.0175 UTC,"[-0.03162849321961403, 0.0018300998490303755, ..."
2,2,2020-03-18 06:39:01.0184 UTC,"[-0.021394122391939163, -0.013377929106354713,..."
3,3,2020-03-18 06:40:19.0195 UTC,"[0.014829986728727818, -0.01779334992170334, 0..."
4,4,2020-03-18 06:40:58.0198 UTC,"[0.013494313694536686, -0.009085348807275295, ..."


### Conversations & Embeddings

In [9]:
# Each conversation grouped into a single thread_id
chats_df = pd.read_csv("../data/chats.csv")
chats_df.head()

,Unnamed: 0,channel_name,thread_id,chat_text
0,0,africa,2022-03-22 19:42:06.219769 UTC,U024WRAA0D9: Hello fellow MLOpsers in Africa :...
1,1,africa,2022-03-24 08:14:33.140029 UTC,U024WRAA0D9: What should our next steps be (fo...
2,2,africa,2022-03-28 11:57:42.840049 UTC,U024WRAA0D9: What’s everyone’s timezone?U024WR...
3,3,africa,2022-04-12 14:36:00.144498 UTC,U03142DQP6Z: Please can we make it later in th...
4,4,africa,2022-04-19 10:24:57.455849 UTC,"U024WRAA0D9: Hello <#C037GTG932B|africa>, I on..."


In [10]:
# The embedding for each conversation with its thread_id (Note: not all embeddings were generated for the chat text)
embeddings_df = pd.read_csv("../data/chats-embeddings-ada-002.csv")
embeddings_df.head()

,Unnamed: 0,thread_id,embedding
0,0,2022-03-22 19:42:06.219769 UTC,"[0.0036350293084979057, -0.01264416053891182, ..."
1,1,2022-03-24 08:14:33.140029 UTC,"[-0.002360287122428417, -0.04199115186929703, ..."
2,2,2022-03-28 11:57:42.840049 UTC,"[0.017543835565447807, 0.0032007887493819, 0.0..."
3,3,2022-04-12 14:36:00.144498 UTC,"[-0.001173610333353281, -0.014446504414081573,..."
4,4,2022-04-19 10:24:57.455849 UTC,"[-0.0025763490702956915, -0.02925489842891693,..."


In [11]:
# Get your embeddings data together.
# Create a temp index of the chats
chats_index = {}
for _, row in tqdm(chats_df.iterrows(), desc="Creating temporary chats index"):
  chats_index[row['thread_id']] = row['chat_text']

# Link the chats and embeddings together
embeddings = []
VECTOR_SIZE = None
for _, row in tqdm(embeddings_df.iterrows(), desc="Collecting chats and embeddings"):
  embedding = json.loads(row['embedding'])
  embeddings.append({"thread_id": row['thread_id'], "embedding":  embedding})
  if not VECTOR_SIZE:
    VECTOR_SIZE = len(embedding)
  else:
    assert VECTOR_SIZE==len(embedding)

Creating temporary chats index: 9719it [00:01, 6558.30it/s]


# Create your vector database
Using https://redis-py.readthedocs.io/en/stable/examples/search_vector_similarity_examples.html

In [12]:
r = redis.StrictRedis.from_url(REDIS_URL)
r.ping()

INDEX_NAME = "vector_index"                              # Vector Index Name
DOC_PREFIX = "DOC:"                                      # RediSearch Key Prefix for the Index
assert VECTOR_SIZE

def create_index(vector_dimensions: int=VECTOR_SIZE):
    try:
        # check to see if index exists
        r.ft(INDEX_NAME).info()
        print("Index already exists!")
    except:
        # schema - we have two fields in our object - thread_id, and embedding
        schema = (
            VectorField("vector",                  # Vector Field Name
                "FLAT", {                          # Vector Index Type: FLAT or HNSW
                    "TYPE": "FLOAT32",             # FLOAT32 or FLOAT64
                    "DIM": vector_dimensions,      # Number of Vector Dimensions
                    "DISTANCE_METRIC": "COSINE",   # Vector Search Distance Metric
                }
            ),
        )

        # index Definition
        definition = IndexDefinition(prefix=[DOC_PREFIX], index_type=IndexType.HASH)

        # create Index
        r.ft(INDEX_NAME).create_index(fields=schema, definition=definition)

In [13]:
# Create the empty index
create_index(vector_dimensions=VECTOR_SIZE)

Index already exists!


In [14]:
# bulk insert data
pipe = r.pipeline()
for row in tqdm(embeddings):
    # define key
    key = f"{DOC_PREFIX}{row['thread_id']}"
    # HSET
    pipe.hset(key, mapping={"vector": np.array(row['embedding']).astype(np.float32).tobytes()})
res = pipe.execute()

100%|██████████| 9713/9713 [00:01<00:00, 4947.77it/s]


In [15]:
def search_index(vector, vector_dimensions: int = VECTOR_SIZE):  
  query = (
    Query("*=>[KNN 2 @vector $vec as score]")
     .sort_by("score")
     .return_fields("id", "score")
     .paging(0, 3)
     .dialect(2)
  )

  query_params = {
      "vec": vector
  }
  return r.ft(INDEX_NAME).search(query, query_params).docs

In [16]:
# test with an existing document, that that document is returned
row = embeddings[2539]
docs = search_index(np.array(row['embedding']).astype(np.float32).tobytes())
assert f"{DOC_PREFIX}{row['thread_id']}"==docs[0].id, "Document does not match"

# Example Q&A with OpenAI

In [17]:
def get_embedding(text):  
  # Use the same embedding generator as what was used on the data!!!
  response = openai.Embedding.create(
    model="text-embedding-ada-002",
    input=text
  )
  return response.data[0].embedding

def summarize(chat_text):
  # Summarize conversations since individually they are long and go over 8k limit
  prompt = "Summarize the following conversation on the MLOps.community slack channel. Do not use the usernames in the summary. ```" + chat_text + "```"
  completion = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[
      {"role": "user", "content": prompt}
    ]
  )
  return completion.choices[0].message.content
  

def extract_answer(chat_texts, question):  
  # Combine the summaries into a prompt and use SotA GPT-4 to answer.
  prompt = "Use the following summaries of conversations on the MLOps.community slack channel backtics to generate an answer for the user question."
  for i, chat_text in enumerate(chat_texts):
    print(f"Getting summary for conversation {i+1}")
    prompt += f"\nConversation {i+1} Summary:\n```\n{summarize(chat_text)}```"

  if not question.endswith("?"):
    question = question + "?"
  prompt+= f"\nQuestion: {question}"
  print(f"Getting answer for the question.")
  completion = openai.ChatCompletion.create(
    model="gpt-4",
    messages=[
      {"role": "user", "content": prompt}
    ]
  )
  content = completion.choices[0].message.content
  return content


In [18]:
def get_answer(question):
  # Get answer to the question by finding the three conversations that are nearest 
  # to the question and then using them to generate the answer.
  print(f"Searching documents nearest to the question.")
  search_vector = np.array(get_embedding(question)).astype(np.float32).tobytes()
  docs = search_index(search_vector)
  # Take the top three answers, and use ChatGPT to form the answer to give the user.
  chat_texts = []
  for doc in docs:
    chat_text = chats_index[doc.id[len(DOC_PREFIX):]]
    chat_texts.append(chat_text)
  if len(chat_texts)>3:
    chat_texts[:3]
  return extract_answer(chat_texts, question)

In [19]:
question="What are some good ways to deploy models on Kubernetes?"
answer = get_answer(question)
print(f"\n\nQuestion: {question}\nAnswer: {answer}")

Searching documents nearest to the question.
Getting summary for conversation 1
Getting summary for conversation 2
Getting answer for the question.


Question: What are some good ways to deploy models on Kubernetes?
Answer: Some good ways to deploy models on Kubernetes include using KServe, Seldon, TensorFlow Serving, and Torchserve. These tools allow you to effectively serve and manage ML models in production. You can also use ONNX Runtime for significant speedups by converting your model checkpoints to ONNX format. Additionally, data orchestration products like Pachyderm, Algorithmia, and Iguazio can be utilized for custom preprocessing and data lineage. When deploying models, it's also a good idea to separate the API/web service from the model inference service for better scalability and use tools like MLflow for user-friendly management.


In [20]:
question="How can I structure a good Data Science team?"
answer = get_answer(question)
print(f"\n\nQuestion: {question}\nAnswer: {answer}")

Searching documents nearest to the question.
Getting summary for conversation 1
Getting summary for conversation 2
Getting answer for the question.


Question: How can I structure a good Data Science team?
Answer: A good Data Science team structure can be achieved by adopting the following strategies:

1. Embed data scientists in product teams: This helps in close collaboration with the product team and enables them to understand the business requirements better.

2. Educate stakeholders: Ensure that stakeholders understand the unpredictable nature of project duration, model accuracy, and business value in data science projects.

3. Iterate and explore: Adopt an iterative and exploratory approach to data science projects, allowing the team to refine their models and solutions over time.

4. Review KPIs in terms of ROI: Regularly review key performance indicators with a focus on return on investment to ensure that data science projects align with business goals.

5. Centralize ML platfo

In [21]:
question="What is the best way to train models for tabular data?"
answer = get_answer(question)
print(f"\n\nQuestion: {question}\nAnswer: {answer}")

Searching documents nearest to the question.
Getting summary for conversation 1
Getting summary for conversation 2
Getting answer for the question.


Question: What is the best way to train models for tabular data?
Answer: The best way to train models for tabular data is often to use boosted decision trees, as they tend to be on par with or outperform neural networks for this type of data. They are also quicker to train, making them suitable for fast iterations. By using smart sampling and limited hyperparameter tuning, trees can achieve high accuracy. However, if you need to blend tabular data with other data types like images or text, using a library like pytorch-widedeep can be effective.
